In [35]:
# All imports and settings

import locale
locale.setlocale(locale.LC_ALL, locale="nl_BE.UTF-8")
locale._override_localeconv["grouping"] = [3, 3, 0] # Hack to show correct thousands separators for nl_BE
def format_int(integer):
  return locale.format_string("%d",integer, grouping=True)

from constants import LIST_GOVERNING_BODY_URIS, PREFIXES_QUERY_PART
from functions import new_sparqlwrapper, execute_query, execute_count_query
from datasource import DATASOURCE
DS = DATASOURCE # Alias
import os, json,re
import matplotlib.pyplot as plt
import numpy as np



In [36]:


# Make labels for all the plots; extend the list
DELETE_WORDS = "de voor"

def transform_label(input):
  output = input.lower()
  for word in DELETE_WORDS.split():
    output = re.sub(
      "\\s+({})\\s+".format(word),
      r" ",
      output,
    )
  output = re.sub(
    r'(\s+)',
    r'-',
    output,
  )
  output = output.replace('é','e')
  return output

LIST_PLOT_IDS = list(map(lambda tup: transform_label(tup[1]),LIST_GOVERNING_BODY_URIS))

EXTENDED_LIST_GOVERNING_BODY_URIS = \
  list(map(
    lambda tup: (*tup[1], LIST_PLOT_IDS[tup[0]],os.getcwd()+"/plots/{}-completeness-count-over-time.png".format(LIST_PLOT_IDS[tup[0]])), 
    zip(range(len(LIST_GOVERNING_BODY_URIS)), 
    LIST_GOVERNING_BODY_URIS)
  ))

# Some month number calculations

# Year, month or tuple (year,month)
def month_nr(a,b=None):
  if b==None and type(a)==tuple:
    return (a[0] - 2010)*12+a[1]-1
  return (a - 2010)*12+b-1

def year_month(monthNr):
  result = monthNr + 2010 * 12
  year = result // 12
  month = result - year * 12 + 1
  return (year,month)

print("Init complete")


Init complete


In [37]:
QUERY_TOTAL_COUNT_TEMPLATE = """\
{prefixes}
SELECT (COUNT(DISTINCT ?res) as ?count) WHERE {{
    ?res a besluit:Besluit.
}}
"""

QUERY_TOTAL_COUNT_DATED_TEMPLATE = """\
{prefixes}
SELECT COUNT(DISTINCT ?res) as ?count WHERE {{
  ?administrativeUnit 
    a besluit:Bestuurseenheid;
    skos:prefLabel ?adminUnitLabel.

  ?governingBodyAbstract
    a besluit:Bestuursorgaan;
    besluit:bestuurt ?administrativeUnit.
  
  {{
    ?session 
      a besluit:Zitting;
      besluit:behandelt ?agendaItem;
      besluit:isGehoudenDoor ?governingBodyAbstract.
  }} UNION {{
    ?session 
      a besluit:Zitting;
      besluit:behandelt ?agendaItem;
      besluit:isGehoudenDoor ?governingBodyTimeSpecified.

    ?governingBodyTimeSpecified 
      a besluit:Bestuursorgaan;
      mandaat:isTijdspecialisatieVan ?governingBodyAbstract.
  }}
  ?agendaItem a besluit:Agendapunt.

  ?agendaItemHandling 
    a besluit:BehandelingVanAgendapunt;
    dct:subject ?agendaItem;
    prov:generated ?res.
  
  ?res a besluit:Besluit;
    eli:date_publication ?publicationDate.
}}
"""

QUERY_TOTAL_COUNT_DATED_ALT_TEMPLATE = """\
{prefixes}
SELECT COUNT(DISTINCT ?res) as ?count WHERE {{
  ?administrativeUnit 
    a besluit:Bestuurseenheid;
    skos:prefLabel ?adminUnitLabel.

  ?governingBodyAbstract
    a besluit:Bestuursorgaan;
    besluit:bestuurt ?administrativeUnit.
  
  {{
    ?session 
      a besluit:Zitting;
      besluit:behandelt ?agendaItem;
      besluit:isGehoudenDoor ?governingBodyAbstract;
      prov:startedAtTime ?startTime.
  }} UNION {{
    ?session 
      a besluit:Zitting;
      besluit:behandelt ?agendaItem;
      besluit:isGehoudenDoor ?governingBodyTimeSpecified;
      prov:startedAtTime ?startTime.

    ?governingBodyTimeSpecified 
      a besluit:Bestuursorgaan;
      mandaat:isTijdspecialisatieVan ?governingBodyAbstract.
  }}
  ?agendaItem a besluit:Agendapunt .

  ?agendaItemHandling 
    a besluit:BehandelingVanAgendapunt;
    dct:subject ?agendaItem;
    prov:generated ?res.
  
  ?res a besluit:Besluit.
}} GROUP BY ?monthNr
"""

QUERY_COMPLETE_EVOLUTION_TOTAL_TEMPLATE = """\
{prefixes}

SELECT ?monthNr ?completeRes ?totalRes {{
  {{
    SELECT ?monthNr (COUNT(DISTINCT ?res1) as ?totalRes) WHERE {{
        ?administrativeUnit 
          a besluit:Bestuurseenheid;
          skos:prefLabel ?adminUnitLabel.

        ?governingBodyAbstract
          a besluit:Bestuursorgaan;
          besluit:bestuurt ?administrativeUnit.
        
        {{
          ?session 
            a besluit:Zitting;
            besluit:behandelt ?agendaItem;
            besluit:isGehoudenDoor ?governingBodyAbstract.
        }} UNION {{
          ?session 
            a besluit:Zitting;
            besluit:behandelt ?agendaItem;
            besluit:isGehoudenDoor ?governingBodyTimeSpecified.

          ?governingBodyTimeSpecified 
            a besluit:Bestuursorgaan;
            mandaat:isTijdspecialisatieVan ?governingBodyAbstract.
        }}
        ?agendaItem a besluit:Agendapunt .

        ?agendaItemHandling 
          a besluit:BehandelingVanAgendapunt;
          dct:subject ?agendaItem;
          prov:generated ?res1.
      
      ?res1 a besluit:Besluit;
        eli:date_publication ?publicationDate1.

      BIND(((year(?publicationDate1) - 2010)*12 + month(?publicationDate1)-1) as ?monthNr)
    }} GROUP BY ?monthNr
  }}
  {{
      SELECT ?monthNr (COUNT(DISTINCT ?res2) as ?completeRes) WHERE {{
        ?administrativeUnit 
          a besluit:Bestuurseenheid;
          skos:prefLabel ?adminUnitLabel.

        ?governingBodyAbstract
          a besluit:Bestuursorgaan;
          besluit:bestuurt ?administrativeUnit.
        
        {{
          ?session 
            a besluit:Zitting;
            besluit:behandelt ?agendaItem;
            besluit:isGehoudenDoor ?governingBodyAbstract.
        }} UNION {{
          ?session 
            a besluit:Zitting;
            besluit:behandelt ?agendaItem;
            besluit:isGehoudenDoor ?governingBodyTimeSpecified.

          ?governingBodyTimeSpecified 
            a besluit:Bestuursorgaan;
            mandaat:isTijdspecialisatieVan ?governingBodyAbstract.
        }}
        ?agendaItem a besluit:Agendapunt .

        ?agendaItemHandling 
          a besluit:BehandelingVanAgendapunt;
          dct:subject ?agendaItem;
          prov:generated ?res2.
      
      ?res2
          a besluit:Besluit;
          eli:date_publication ?publicationDate2;
          eli:description ?description;
          prov:value ?value;
          eli:title ?title.
    
      FILTER(
        datatype(?description) = xsd:string &&
        isLiteral(?value) &&
        datatype(?title) = xsd:string
      )

      BIND(((year(?publicationDate2) - 2010)*12 + month(?publicationDate2)-1) as ?monthNr)
    }} GROUP BY ?monthNr
  }}
}} ORDER BY ?monthNr
"""

QUERY_COMPLETE_EVOLUTION_TOTAL_ALT_TEMPLATE = """\
{prefixes}

SELECT ?monthNr ?completeRes ?totalRes {{
  {{
    SELECT ?monthNr (COUNT(DISTINCT ?res1) as ?totalRes) WHERE {{
        ?administrativeUnit 
          a besluit:Bestuurseenheid;
          skos:prefLabel ?adminUnitLabel.

        ?governingBodyAbstract
          a besluit:Bestuursorgaan;
          besluit:bestuurt ?administrativeUnit.
        
        {{
          ?session 
            a besluit:Zitting;
            besluit:behandelt ?agendaItem;
            besluit:isGehoudenDoor ?governingBodyAbstract;
            prov:startedAtTime ?startTime1.
        }} UNION {{
          ?session 
            a besluit:Zitting;
            besluit:behandelt ?agendaItem;
            besluit:isGehoudenDoor ?governingBodyTimeSpecified;
            prov:startedAtTime ?startTime1.

          ?governingBodyTimeSpecified 
            a besluit:Bestuursorgaan;
            mandaat:isTijdspecialisatieVan ?governingBodyAbstract.
        }}
        ?agendaItem a besluit:Agendapunt .

        ?agendaItemHandling 
          a besluit:BehandelingVanAgendapunt;
          dct:subject ?agendaItem;
          prov:generated ?res1.
      
      ?res1 a besluit:Besluit;
        eli:date_publication ?publicationDate1.

      BIND(((year(?startTime1) - 2010)*12 + month(?startTime1)-1) as ?monthNr)
    }} GROUP BY ?monthNr
  }}
  {{
      SELECT ?monthNr (COUNT(DISTINCT ?res2) as ?completeRes) WHERE {{
        ?administrativeUnit 
          a besluit:Bestuurseenheid;
          skos:prefLabel ?adminUnitLabel.

        ?governingBodyAbstract
          a besluit:Bestuursorgaan;
          besluit:bestuurt ?administrativeUnit.
        
        {{
          ?session 
            a besluit:Zitting;
            besluit:behandelt ?agendaItem;
            besluit:isGehoudenDoor ?governingBodyAbstract;
            prov:startedAtTime ?startTime2.
        }} UNION {{
          ?session 
            a besluit:Zitting;
            besluit:behandelt ?agendaItem;
            besluit:isGehoudenDoor ?governingBodyTimeSpecified;
            prov:startedAtTime ?startTime2.

          ?governingBodyTimeSpecified 
            a besluit:Bestuursorgaan;
            mandaat:isTijdspecialisatieVan ?governingBodyAbstract.
        }}
        ?agendaItem a besluit:Agendapunt .

        ?agendaItemHandling 
          a besluit:BehandelingVanAgendapunt;
          dct:subject ?agendaItem;
          prov:generated ?res2.
      
      ?res2
          a besluit:Besluit;
          eli:date_publication ?publicationDate2;
          eli:description ?description;
          prov:value ?value;
          eli:title ?title.
    
      FILTER(
        datatype(?description) = xsd:string &&
        isLiteral(?value) &&
        datatype(?title) = xsd:string
      )

      BIND(((year(?startTime2) - 2010)*12 + month(?startTime2)-1) as ?monthNr)
    }} GROUP BY ?monthNr
  }}
}} ORDER BY ?monthNr
"""

QUERY_COMPLETE_EVOLUTION_BO_TEMPLATE = """\
{prefixes}

SELECT ?monthNr ?completeRes ?totalRes {{
  {{
    SELECT ?monthNr (COUNT(DISTINCT ?res1) as ?totalRes) WHERE {{
        ?administrativeUnit 
          a besluit:Bestuurseenheid;
          skos:prefLabel ?adminUnitLabel.

        ?governingBodyAbstract
          a besluit:Bestuursorgaan;
          besluit:bestuurt ?administrativeUnit;
          org:classification <{governing_body_uri}>.
        
        {{
          ?session 
            a besluit:Zitting;
            besluit:behandelt ?agendaItem;
            besluit:isGehoudenDoor ?governingBodyAbstract.
        }} UNION {{
          ?session 
            a besluit:Zitting;
            besluit:behandelt ?agendaItem;
            besluit:isGehoudenDoor ?governingBodyTimeSpecified.

          ?governingBodyTimeSpecified 
            a besluit:Bestuursorgaan;
            mandaat:isTijdspecialisatieVan ?governingBodyAbstract.
        }}
        ?agendaItem a besluit:Agendapunt .

        ?agendaItemHandling 
          a besluit:BehandelingVanAgendapunt;
          dct:subject ?agendaItem;
          prov:generated ?res1.
      
      ?res1 a besluit:Besluit;
        eli:date_publication ?publicationDate1.

      BIND(((year(?publicationDate1) - 2010)*12 + month(?publicationDate1)-1) as ?monthNr)
    }} GROUP BY ?monthNr
  }}
  {{
      SELECT ?monthNr (COUNT(DISTINCT ?res2) as ?completeRes) WHERE {{
        ?administrativeUnit 
          a besluit:Bestuurseenheid;
          skos:prefLabel ?adminUnitLabel.

        ?governingBodyAbstract
          a besluit:Bestuursorgaan;
          besluit:bestuurt ?administrativeUnit;
          org:classification <{governing_body_uri}>.
        
        {{
          ?session 
            a besluit:Zitting;
            besluit:behandelt ?agendaItem;
            besluit:isGehoudenDoor ?governingBodyAbstract.
        }} UNION {{
          ?session 
            a besluit:Zitting;
            besluit:behandelt ?agendaItem;
            besluit:isGehoudenDoor ?governingBodyTimeSpecified.

          ?governingBodyTimeSpecified 
            a besluit:Bestuursorgaan;
            mandaat:isTijdspecialisatieVan ?governingBodyAbstract.
        }}
        ?agendaItem a besluit:Agendapunt .

        ?agendaItemHandling 
          a besluit:BehandelingVanAgendapunt;
          dct:subject ?agendaItem;
          prov:generated ?res2.
      
      ?res2
          a besluit:Besluit;
          eli:date_publication ?publicationDate2;
          eli:description ?description;
          prov:value ?value;
          eli:title ?title.
    
      FILTER(
        datatype(?description) = xsd:string &&
        isLiteral(?value) &&
        datatype(?title) = xsd:string
      )

      BIND(((year(?publicationDate2) - 2010)*12 + month(?publicationDate2)-1) as ?monthNr)
    }} GROUP BY ?monthNr
  }}
}} ORDER BY ?monthNr
"""
QUERY_COMPLETE_EVOLUTION_BO_ALT_TEMPLATE = """\
{prefixes}

SELECT ?monthNr ?completeRes ?totalRes {{
  {{
    SELECT ?monthNr (COUNT(DISTINCT ?res1) as ?totalRes) WHERE {{
        ?administrativeUnit 
          a besluit:Bestuurseenheid;
          skos:prefLabel ?adminUnitLabel.

        ?governingBodyAbstract
          a besluit:Bestuursorgaan;
          besluit:bestuurt ?administrativeUnit;
          org:classification <{governing_body_uri}>.
        
        {{
          ?session 
            a besluit:Zitting;
            besluit:behandelt ?agendaItem;
            besluit:isGehoudenDoor ?governingBodyAbstract;
            prov:startedAtTime ?startTime1.
        }} UNION {{
          ?session 
            a besluit:Zitting;
            besluit:behandelt ?agendaItem;
            besluit:isGehoudenDoor ?governingBodyTimeSpecified;
            prov:startedAtTime ?startTime1.

          ?governingBodyTimeSpecified 
            a besluit:Bestuursorgaan;
            mandaat:isTijdspecialisatieVan ?governingBodyAbstract.
        }}
        ?agendaItem a besluit:Agendapunt .

        ?agendaItemHandling 
          a besluit:BehandelingVanAgendapunt;
          dct:subject ?agendaItem;
          prov:generated ?res1.
      
      ?res1 a besluit:Besluit.

      BIND(((year(?startTime1) - 2010)*12 + month(?startTime1)-1) as ?monthNr)
    }} GROUP BY ?monthNr
  }}
  {{
      SELECT ?monthNr (COUNT(DISTINCT ?res2) as ?completeRes) WHERE {{
        ?administrativeUnit 
          a besluit:Bestuurseenheid;
          skos:prefLabel ?adminUnitLabel.

        ?governingBodyAbstract
          a besluit:Bestuursorgaan;
          besluit:bestuurt ?administrativeUnit;
          org:classification <{governing_body_uri}>.
        
        {{
          ?session 
            a besluit:Zitting;
            besluit:behandelt ?agendaItem;
            besluit:isGehoudenDoor ?governingBodyAbstract;
            prov:startedAtTime ?startTime2.
        }} UNION {{
          ?session 
            a besluit:Zitting;
            besluit:behandelt ?agendaItem;
            besluit:isGehoudenDoor ?governingBodyTimeSpecified;
            prov:startedAtTime ?startTime2.

          ?governingBodyTimeSpecified 
            a besluit:Bestuursorgaan;
            mandaat:isTijdspecialisatieVan ?governingBodyAbstract.
        }}
        ?agendaItem a besluit:Agendapunt .

        ?agendaItemHandling 
          a besluit:BehandelingVanAgendapunt;
          dct:subject ?agendaItem;
          prov:generated ?res2.
      
      ?res2
          a besluit:Besluit;
          eli:description ?description;
          prov:value ?value;
          eli:title ?title.
    
      FILTER(
        datatype(?description) = xsd:string &&
        isLiteral(?value) &&
        datatype(?title) = xsd:string
      )

      BIND(((year(?startTime2) - 2010)*12 + month(?startTime2)-1) as ?monthNr)
    }} GROUP BY ?monthNr
  }}
}} ORDER BY ?monthNr
"""
# print("Query loaded")
example = QUERY_COMPLETE_EVOLUTION_BO_TEMPLATE.format(
  governing_body_uri="example",
  prefixes=PREFIXES_QUERY_PART,
)
print(example)

PREFIX besluittype: <https://data.vlaanderen.be/id/concept/BesluitType/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX mu: <http://mu.semte.ch/vocabularies/core/>
PREFIX task: <http://redpencil.data.gift/vocabularies/tasks/>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX nie: <http://www.semanticdesktop.org/ontologies/2007/01/19/nie#>
PREFIX ext: <http://mu.semte.ch/vocabularies/ext/>
PREFIX oslc: <http://open-services.net/ns/core#>
PREFIX cogs: <http://vocab.deri.ie/cogs#>
PREFIX adms: <http://www.w3.org/ns/adms#>
PREFIX nfo: <http://www.semanticdesktop.org/ontologies/2007/03/22/nfo#>
PREFIX dbpedia: <http://dbpedia.org/resource/>
PREFIX besluit: <http://data.vlaanderen.be/ns/besluit#>
PREFIX besluitvor: <https://data.vlaanderen.be/ns/besluitvorming#>
PREFIX generiek: <https://data.vlaanderen.be/ns/generiek#>
PREFIX dossier: 

In [38]:
from matplotlib.ticker import MultipleLocator, FormatStrFormatter

# CONSTANTS
i1 = month_nr((2019,1)) # From 2019 january
i2 = month_nr((2024,3)) # Up to but not including 2024 March

months = i2 - i1

#Generate labels for plots
labels = []

for i in range(i1,i2):
  labels.append("{}-{}".format(*year_month(i)))

def results_to_lists(result):
  complete = np.zeros(months, dtype="int")
  total = np.zeros(months, dtype="int")
  not_complete = np.zeros(months, dtype="int")
  # Output processing
  for row in result:
    total_res = int(row['totalRes']['value'])
    complete_res = int(row['completeRes']['value'])
    i = int(row['monthNr']['value'])
    if i1 <= i and i < i2:
      not_complete[i-i1] = total_res - complete_res
      complete[i-i1] = complete_res
      total[i-i1] = total_res
  return complete, not_complete, total

def generate_plot(complete, not_complete, total,title=None,filename=None,cumulative=False):
  if filename == None:
    raise TypeError("'filename' is a required kwarg")
  if title == None:
    raise TypeError("'title' is a required kwarg")
  c = np.cumsum(complete) if cumulative else complete
  nc = np.cumsum(not_complete) if cumulative else not_complete

  fig, ax = plt.subplots()
  p1 = ax.bar(labels, c, label="Matuur")
  p2 = ax.bar(labels, nc, label="Niet matuur")
  ax.set_title(title)
  majorLocator = MultipleLocator(6)
  ax.xaxis.set_major_locator(majorLocator)
  plt.xticks(rotation=45)

  ax.legend(loc="upper left")

  plt.savefig(filename)
  plt.close()

print("Processing and plot generation functions loaded")

Processing and plot generation functions loaded


In [39]:
# Plot generation and querying. Will take a long time

sparql = new_sparqlwrapper(DS["endpoint"],user=DS["user"],password=DS["password"])

#Counts
print("Performing count queries")
total_query = QUERY_TOTAL_COUNT_TEMPLATE.format(
  prefixes=PREFIXES_QUERY_PART
)
total_count = execute_count_query(sparql,total_query)
total_dated_query = QUERY_TOTAL_COUNT_DATED_ALT_TEMPLATE.format(
  prefixes=PREFIXES_QUERY_PART
)
total_dated_count = execute_count_query(sparql,total_dated_query)

print("Total: {}, dated:{}, coverage:{:2.2%}".format(total_count,total_dated_count,total_dated_count / total_count))

#First perform query for all resolutuons
print("Generating plot for ALL")
query = QUERY_COMPLETE_EVOLUTION_TOTAL_ALT_TEMPLATE.format(
  prefixes=PREFIXES_QUERY_PART,
)
result = execute_query(sparql, query)
lists = results_to_lists(result)

print("\tQuery succeeded")
all_filename = os.getcwd()+"/plots/all-completeness-count-over-time.png"
generate_plot(
  *lists,
  title="Evolutie van maturiteit voor alle besluiten",
  filename=all_filename,
  cumulative=False,
)
print("\tPlot generated: ALL")

# Now perform queries for all BO's
# A row in the extended list is [uri,label,id_string,plot_file_name]
for parameters in EXTENDED_LIST_GOVERNING_BODY_URIS:

  print("Generating for'{}'({})".format(*parameters[1:3]))
  query = QUERY_COMPLETE_EVOLUTION_BO_ALT_TEMPLATE.format(
    governing_body_uri=parameters[0],
    prefixes=PREFIXES_QUERY_PART,
  )
  result = execute_query(sparql, query)
  lists = results_to_lists(result)

  print("\tQuery succeeded")
  generate_plot(
    *lists,
    title="Evolutie van maturiteit voor {}".format(parameters[1][:32]),
    filename=parameters[3],
    cumulative=False,
  )
  print("\tPlot generated:{}".format(parameters[3]))

# Add the all plot to te extended list for report generation. At the beginning
EXTENDED_LIST_GOVERNING_BODY_URIS.insert(0,
  [None,"Alles","all",all_filename]
)

Performing count queries
Total: 2434838, dated:1705142, coverage:70.03%
Generating plot for ALL
	Query succeeded
	Plot generated: ALL
Generating for'Raad van bestuur'(raad-van-bestuur)
	Query succeeded
	Plot generated:/Users/dv/Documents/abb/rapport-20240403/plots/raad-van-bestuur-completeness-count-over-time.png
Generating for'Bevoegd beslissingsorgaan'(bevoegd-beslissingsorgaan)
	Query succeeded
	Plot generated:/Users/dv/Documents/abb/rapport-20240403/plots/bevoegd-beslissingsorgaan-completeness-count-over-time.png
Generating for'Adjunct-algemeen directeur'(adjunct-algemeen-directeur)
	Query succeeded
	Plot generated:/Users/dv/Documents/abb/rapport-20240403/plots/adjunct-algemeen-directeur-completeness-count-over-time.png
Generating for'Regionaal bestuurscomité'(regionaal-bestuurscomite)
	Query succeeded
	Plot generated:/Users/dv/Documents/abb/rapport-20240403/plots/regionaal-bestuurscomite-completeness-count-over-time.png
Generating for'Gouverneur'(gouverneur)
	Query succeeded
	Plot

In [40]:
from itertools import batched
import datetime
# Markdown file generation

FILE_TEMPLATE = """\
@import "rapport.less"

# {title}

<div class="contactinfo">

For questions and requests
  * ✉️ dennis.van.eecke@codifly.be
  * Rocket Chat: `dennisve`
  * Matrix: `@dennisvaneecke:matrix.org`

</div>

Date generated: {date}
Version: {version}
Dataset: Lokaal beslist production data dump from March 2024

## Description
{description}

| Totaal # besluiten | Totaal # gedateerde besluiten |
| :---: | :---: |
| {total_count} | {total_dated_count} ({total_dated_percent:2.2%}) |

## Results

This is the automatically generated result:

{table}
"""

TABLE_TEMPLATE = """\
<table class="plot-table-two-col">
  <tbody>
    {rows}
  <tbody>
</table>
"""

TABLE_ROW_TEMPLATE = """\
    <tr>
      {cells}
    </tr>
"""

TABLE_CELL_TEMPLATE = """\
      <td>
        <p>{governing_body_name}</p>
        <img src="{relative_path_image}">
      </td>
"""

def gov_body_param_to_cell(param):
  return TABLE_CELL_TEMPLATE.format(
    governing_body_name=param[1],
    relative_path_image="plots/{}-completeness-count-over-time.png".format(param[2])
  )

cells = list(map(gov_body_param_to_cell,EXTENDED_LIST_GOVERNING_BODY_URIS))

def cells_to_rows(row_cells):
  return TABLE_ROW_TEMPLATE.format(
    cells="".join(row_cells).strip()
  )

rows = list(map(cells_to_rows, batched(cells,n=2)))

table = TABLE_TEMPLATE.format(
  rows="".join(rows).strip()
)
today = datetime.datetime.now()
file_contents = FILE_TEMPLATE.format(
  title="Evolution of resolution maturity over time",
  date=f"{today:%B %d, %Y %H:%m}",
  version="1.1",
  description="""\
This report contains plots for each of the different types of governing bodies. The X-axis consists of months, the Y-axis is an amount of resolutions.\
This report catches only the resolutions which can be dated by regaring the date of the session and are correctly linked in an OSLO compatible way. This is about {total_dated_percent:2.2%} of all resolutions in the dataset.\
The incorrectly linked resolutions are not considered in this analysis but their source might be the subject of future anlyses.\
The plots distinguish 'mature' and 'not mature' resolution records. Mature means the resolutions that are linked to a description, a value and a title and that the objects associated with these predicates are strings.\
Immature resulutions are all resolutions which do not adhere to this standard.
""".format(
  total_dated_percent = total_dated_count / total_count,
),
  table=table,
  total_count=format_int(total_count),
  total_dated_count=format_int(total_dated_count),
  total_dated_percent = total_dated_count / total_count,
)

with open(os.getcwd()+"/overview-all-maturity-evolutions.md","w",encoding="utf-8") as file:
  file.write(file_contents)

print("Output markdown table generated")


Output markdown table generated
